### Importando as blibliotecas 

In [15]:
import pandas as pd
import numpy as np

In [16]:

## Importando os modelos 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

In [17]:
from sklearn.pipeline import Pipeline, make_pipeline # carregar as pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder # transformar as class do df como numerica
from sklearn.compose import ColumnTransformer # transformacoes especifica das colunas formatando os dados
from sklearn.impute import KNNImputer, SimpleImputer # KNN para inputar dados faltantes
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV 

In [18]:
train = pd.read_csv('D:\\Projeto_\\ModelMagicPipeline\\data\\train.csv')
test = pd.read_csv('D:\\Projeto_\\ModelMagicPipeline\\data\\test.csv')

In [19]:
display(train)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [20]:
display(test)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


#### Iniciar primeiro separando os dados (Pontos importantes)
* Pegar as variaveis de treino e separar para validação
* Separar o teste para usar la no final
* Seleciona o melhor modelo dentro das validações
* Para depois usar os dados de treinamento


#### Depois ter limpados os dados ter estruturado todos eles
*   Então podemos separa em variaveis categoricas e numericas

In [21]:
train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [22]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
catecoric_features = [col for col in train.columns if train[col].dtype.name == 'object']
catecoric_features

['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

In [24]:
numeric_features = [col for col in train.columns if col not in catecoric_features]
numeric_features

['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [25]:
_ = []
for i in numeric_features:
    if i == 'Survived':
        continue
    else : 
        _.append(i)
numeric_features = _
numeric_features

['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']

In [26]:
## Aqui estou testando a pípeline para iniciala

## pegando so os modelos que me interesao para colocar na pipeline de teste.

## Aqui to criando os transformadores numericos
## Aqui tambem adicionei oimputer simples para adicionar as nos valores NaN
numeric_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler(with_mean=False)) 
])

## Aqui to criando os transformadores não numericos
catecoric_transform = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

## Aqui to unindo todos com 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transform, numeric_features),
        ('cat', catecoric_transform, catecoric_features)])

In [27]:
from sklearn.ensemble import VotingClassifier


## Aqui crie pipelines separadas para cada modelo pos a lib do sklearn não premite usar em um unica sem ser no Vorting
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier())
])

knn_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

dt_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier())
])


classifiers = [
    ('RandomForest', rf_pipeline),
    ('KNeighbors', knn_pipeline),
    ('LogisticRegression', lr_pipeline),
    ('DecisionTree', dt_pipeline)
]

voting_classifier = VotingClassifier(estimators=classifiers)

In [28]:
## Aqui temos toda ela pronta

voting_classifier

VotingClassifier(estimators=[('RandomForest',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='median')),
                                                                                                ('scaler',
                                                                                                 StandardScaler(with_mean=False))]),
                                                                                ['PassengerId',
                                                                                 'Pclass',
                                                                                 'Age',
                                                 

#### Agora posso separar meu treino 

In [29]:
X = train.drop('Survived', axis=1)
y = train['Survived']

X_train, X_valid, y_train, y_valid = train_test_split(X, y)


In [30]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((668, 11), (223, 11), (668,), (223,))

In [31]:
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('RandomForest',
                              Pipeline(steps=[('preprocessor',
                                               ColumnTransformer(transformers=[('num',
                                                                                Pipeline(steps=[('imputer',
                                                                                                 SimpleImputer(strategy='median')),
                                                                                                ('scaler',
                                                                                                 StandardScaler(with_mean=False))]),
                                                                                ['PassengerId',
                                                                                 'Pclass',
                                                                                 'Age',
                                                 

In [32]:
voting_classifier.score(X_valid, y_valid)

0.8475336322869955

#### O KFold não consigo usar dentro do pipeline (Muito cuidado!)
* Porem conseguimos usar ele depois 

In [33]:
validation_cruz = KFold(n_splits=10, shuffle=True)
validation_cruz

KFold(n_splits=10, random_state=None, shuffle=True)

In [34]:
acc_mean_all = cross_val_score(voting_classifier, X_valid, y_valid, cv=validation_cruz)
acc_mean_all

array([0.91304348, 0.86956522, 0.82608696, 0.77272727, 0.72727273,
       0.77272727, 0.77272727, 0.95454545, 0.90909091, 0.68181818])

In [35]:
for name, clf in classifiers:
    scores = cross_val_score(clf, X_train, y_train, cv=validation_cruz)  # Use validação cruzada k-fold (cv=5 como exemplo)
    print(f'{name}: Scores de Validação Cruzada: {scores}')
    print(f'{name}: Média dos Scores de Validação Cruzada: {scores.mean()}')

RandomForest: Scores de Validação Cruzada: [0.73134328 0.85074627 0.82089552 0.76119403 0.8358209  0.85074627
 0.8358209  0.74626866 0.86363636 0.83333333]
RandomForest: Média dos Scores de Validação Cruzada: 0.8129805517865221
KNeighbors: Scores de Validação Cruzada: [0.85074627 0.79104478 0.74626866 0.76119403 0.7761194  0.74626866
 0.74626866 0.73134328 0.8030303  0.75757576]
KNeighbors: Média dos Scores de Validação Cruzada: 0.7709859791949344


d:\Users\Renan Lemes\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression: Scores de Validação Cruzada: [0.8358209  0.73134328 0.79104478 0.71641791 0.80597015 0.7761194
 0.85074627 0.76119403 0.8030303  0.81818182]
LogisticRegression: Média dos Scores de Validação Cruzada: 0.7889868837630031
DecisionTree: Scores de Validação Cruzada: [0.74626866 0.79104478 0.8358209  0.79104478 0.73134328 0.88059701
 0.79104478 0.89552239 0.75757576 0.83333333]
DecisionTree: Média dos Scores de Validação Cruzada: 0.805359565807327


#### Agora com os modelos treinados podemos então salvar eles em um json


In [36]:
import joblib
import json
import pickle

In [37]:
model1 = rf_pipeline
model2 = dt_pipeline


model_serialize_rf = joblib.dump(model1, '..\\model\\RandomForest.joblib')
model_serialize_dt = joblib.dump(model2, '..\\model\\DecisionTree.joblib')

model_info_rf = {
    'model_type': 'RandomForest',
    'model_params': model1.get_params(),  # Pode variar dependendo do modelo
    'other_info': 'Informações adicionais se eu quiser'
}

model_info_dt = {
    'model_type': 'DecisionTree',
    'model_params': model2.get_params(),  # Pode variar dependendo do modelo
    'other_info': 'Informações adicionais se eu quiser'
}


#### Outra forma de salvar os modelo de ML são PKL

In [38]:
joblib.dump(model1, '..\\model\\RandomForest.pkl')
joblib.dump(model1, '..\\model\\DecisionTree.pkl')


['..\\model\\DecisionTree.pkl']

In [39]:
### Para carregar o modelo treinado ai usamos 
model1 = joblib.load("..\\model\\RandomForest.pkl")

In [40]:
model1

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False))]),
                                                  ['PassengerId', 'Pclass',
                                                   'Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant